<a href="https://colab.research.google.com/github/sebavassou/SIG/blob/main/SIG_Enap_aula_06_construindo_BD_SIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalar as bibliotecas de SIG
*Ambiente Linux*
1.   Atualiza o repositório local (sincroniza) e depois o sistema  - ***'apt'***
2.   Instala as dependências para as bibliotecas Python - ***'apt'***
3.   Atualiza o ***'pip'*** (instalado das bibliotecas python)
4.   Instala as bibliotecas de Python para SIG

Obs.:
- O Google Colab já possui alguma das bibliotecas
- Algumas bibliotecas depende de outras (instala automáticamente)

In [ ]:
!apt update
!apt upgrade
!apt install libspatialindex-dev
!apt install libgdal-dev libgdal-pcraster-dev libgdal-perl
!apt install libsqlite3-mod-spatialite
!apt install gdal-bin

### Arquivos para Download
*   Setores (Shapefile)  
https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2022/setores/shp/BR/BR_setores_CD2022.zip
*    Agregados_por_setores_caracteristicas_domicilio1_BR  
https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/Agregados_por_Setor_csv/Agregados_por_setores_caracteristicas_domicilio1_BR.zip  
*    Dicionário
https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/dicionario_de_dados_agregados_por_setores_censitarios_20250417.xlsx

In [ ]:
!wget https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2022/setores/shp/BR/BR_setores_CD2022.zip
!wget https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/Agregados_por_Setor_csv/Agregados_por_setores_caracteristicas_domicilio1_BR.zip
!wget https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/dicionario_de_dados_agregados_por_setores_censitarios_20250417.xlsx

!unzip BR_setores_CD2022.zip
!unzip Agregados_por_setores_caracteristicas_domicilio1_BR.zip

In [ ]:
!ls -1

In [ ]:
!head Agregados_por_setores_caracteristicas_domicilio1_BR.csv

In [ ]:
# Como saber se o Shapefile possui Sistema de Referência de Coordenadas ?

### Criar o Geopackage e inserir as informações
*   BR_setores_CD2022 (Shapefile)  
*   Agregados_por_setores_caracteristicas_domicilio1_BR (CSV)
*   dicionario_de_dados_agregados_por_setores_censitarios_20250417 (Excel)


#### BR_setores_CD2022 (Shapefile)  - Cria um Geopackage

In [ ]:
!ogr2ogr -f "GPKG" enap_sig_2025_06_25.gpkg BR_setores_CD2022.shp -nln setores_cd2022

#### Agregados_por_setores_caracteristicas_domicilio1_BR (CSV) - **Adiciona**

In [ ]:
!ogr2ogr -f "GPKG" -update enap_sig_2025_06_25.gpkg Agregados_por_setores_caracteristicas_domicilio1_BR.csv -nln agregados_por_setores -nlt NONE

#### dicionario_de_dados_agregados_por_setores_censitarios_20250417 (Excel)

In [ ]:
import pandas as pd

filepath = 'dicionario_de_dados_agregados_por_setores_censitarios_20250417.xlsx'

with pd.ExcelFile( filepath ) as xls:
    print('\n'.join( xls.sheet_names ) )


In [ ]:
df = pd.read_excel( filepath, sheet_name=2)
df.head()

In [ ]:
import sqlite3
from datetime import datetime

filepath = 'enap_sig_2025_06_25.gpkg'
table_name = 'dic_vars'

with sqlite3.connect( filepath) as conn:
    df.to_sql( table_name, conn, if_exists='replace', index=False )
    # Insert table in gpkg_contents (Geopackage)
    cursor = conn.cursor()
    query = """
    INSERT INTO gpkg_contents (
        table_name, data_type, identifier, description,
        last_change, min_x, min_y, max_x, max_y, srs_id
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    cursor.execute(
        query,
        (
            table_name, 'attributes', table_name, '',
            datetime.utcnow().isoformat(), None, None, None, None, None
        )
    )


### Verificação das tabelas/camadas

In [ ]:
import pandas as pd
import sqlite3

filepath = 'enap_sig_2025_06_25.gpkg'
query = "SELECT table_name FROM gpkg_contents"
#query = "SELECT name FROM sqlite_master WHERE type='table';"

with sqlite3.connect( filepath) as conn:
    tables = pd.read_sql_query( query, conn )
    print( tables )

In [ ]:
!ogrinfo -so -al enap_sig_2025_06_25.gpkg